<a href="https://colab.research.google.com/github/LuisFernandoCampos/modelo_renovacion_prestamo/blob/main/modelo_renovacion_credito_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [354]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn import metrics
from sklearn.metrics import confusion_matrix

from xgboost import XGBClassifier

In [303]:
rp=pd.read_csv("/content/renovacion_prestamo.csv", sep=";")

# 1.- ANALISIS EXPLORATORIO DE DATOS

In [304]:
rp.head(2)

,MES,CLIENTE,LINEA_RENOVADO,PLAZO_RENOVADO,FLAG_VENTA,USO_LINEA_TOTAL_TC_T2,USO_TRIM_LINEA_BBVA,NR_ENTIDADES_TOTAL_T2,DIFF_NRO_ENTIDA_TOTALES_T2_T12,SDO_CONSUMO_T2,...,PConsumo_Sldo_Bco_T1,SDO_BCO_tot_sm_pasivo_Bco_6M,EDAD,SEXO,EST_CIVIL,ANTIGUEDAD_MES,REGION,FLAG_LIMA_PROVINCIA,SUELDO_ESTIMADO,CUBRIR_DEUDA_CONSUMO_SF_RENOVA_PLD
0,201506,1,3770,12,0,0.0,0.0,1,-1,271.12,...,457,3754.333333,25.0,M,S,23.0,LIMA NORTE,1,3531.0,0.071915
1,201506,2,1300,6,0,0.0,0.0,1,0,11200.00,...,11200,1581.333333,33.0,M,S,56.0,LIMA ESTE,1,1322.0,8.615380


In [305]:
rp["FLAG_VENTA"].value_counts()/len(rp)*100

0    96.011695
1     3.988305
Name: FLAG_VENTA, dtype: float64

In [306]:
rp["CLIENTE"].count()

87556

In [307]:
print("tamaño: ",rp.shape)

tamaño:  (87556, 22)


In [308]:
rp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87556 entries, 0 to 87555
Data columns (total 22 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   MES                                 87556 non-null  int64  
 1   CLIENTE                             87556 non-null  int64  
 2   LINEA_RENOVADO                      87556 non-null  int64  
 3   PLAZO_RENOVADO                      87556 non-null  int64  
 4   FLAG_VENTA                          87556 non-null  int64  
 5   USO_LINEA_TOTAL_TC_T2               68364 non-null  float64
 6   USO_TRIM_LINEA_BBVA                 68364 non-null  float64
 7   NR_ENTIDADES_TOTAL_T2               87556 non-null  int64  
 8   DIFF_NRO_ENTIDA_TOTALES_T2_T12      87556 non-null  int64  
 9   SDO_CONSUMO_T2                      82370 non-null  float64
 10  RESENCIA_OFERTA_PLD_RENOVADO        73700 non-null  float64
 11  Ahorro_Sldo_Bco_T1                  87556

In [309]:
rp.isna().sum()

MES                                       0
CLIENTE                                   0
LINEA_RENOVADO                            0
PLAZO_RENOVADO                            0
FLAG_VENTA                                0
USO_LINEA_TOTAL_TC_T2                 19192
USO_TRIM_LINEA_BBVA                   19192
NR_ENTIDADES_TOTAL_T2                     0
DIFF_NRO_ENTIDA_TOTALES_T2_T12            0
SDO_CONSUMO_T2                         5186
RESENCIA_OFERTA_PLD_RENOVADO          13856
Ahorro_Sldo_Bco_T1                        0
PConsumo_Sldo_Bco_T1                      0
SDO_BCO_tot_sm_pasivo_Bco_6M              0
EDAD                                      1
SEXO                                      1
EST_CIVIL                                 1
ANTIGUEDAD_MES                            1
REGION                                   18
FLAG_LIMA_PROVINCIA                       0
SUELDO_ESTIMADO                          37
CUBRIR_DEUDA_CONSUMO_SF_RENOVA_PLD        0
dtype: int64

In [310]:
# Filtrar columnas con valores nulos
columnas_con_nulos = rp.columns[rp.isna().sum() > 0]
p50=lambda x: x.quantile(0.50)
p99=lambda x: x.quantile(0.99)
p95=lambda x: x.quantile(0.95)
coeficiente_variabilidad= (rp[columnas_con_nulos].std() / rp[columnas_con_nulos].mean())*100

<ipython-input-310-cc6be0556025>:6: FutureWarning: The default value of numeric_only in DataFrame.std is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  coeficiente_variabilidad= (rp[columnas_con_nulos].std() / rp[columnas_con_nulos].mean())*100
<ipython-input-310-cc6be0556025>:6: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  coeficiente_variabilidad= (rp[columnas_con_nulos].std() / rp[columnas_con_nulos].mean())*100


In [311]:
coeficiente_variabilidad.sort_values(ascending=False)

USO_TRIM_LINEA_BBVA             211.180842
SDO_CONSUMO_T2                  149.946500
USO_LINEA_TOTAL_TC_T2           149.463004
SUELDO_ESTIMADO                 125.966387
ANTIGUEDAD_MES                   60.225762
RESENCIA_OFERTA_PLD_RENOVADO     40.323096
EDAD                             24.697052
dtype: float64

In [312]:
rp[columnas_con_nulos].agg(["mean", p50, "min", "max",p95,p99]).T


<ipython-input-312-62699aac1bf3>:1: FutureWarning: ['SEXO', 'EST_CIVIL', 'REGION'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  rp[columnas_con_nulos].agg(["mean", p50, "min", "max",p95,p99]).T


,mean,<lambda>,min,max,<lambda>,<lambda>
USO_LINEA_TOTAL_TC_T2,0.128593,0.030634,0.0,1.466976,0.556814,0.809457
USO_TRIM_LINEA_BBVA,0.096935,0.000000,0.0,1.666666,0.605864,0.900713
SDO_CONSUMO_T2,14385.172299,8368.145000,0.0,722781.090000,48501.905000,107020.912400
RESENCIA_OFERTA_PLD_RENOVADO,14.704464,15.000000,1.0,21.000000,21.000000,21.000000
EDAD,43.820753,43.000000,21.0,72.000000,62.000000,65.000000
ANTIGUEDAD_MES,124.336508,106.000000,3.0,561.000000,241.000000,294.000000
SUELDO_ESTIMADO,4766.287255,3178.120000,0.0,240909.000000,12746.300000,24750.000000


# 2.- PREPROCESAMIENTO

  # 2.1.- LIMPIEZA DE DATOS

# COLUMNAS NUMERICAS


**USO_LINEA_TOTAL_TC_T2**

In [313]:
#sns.histplot(rp["USO_LINEA_TOTAL_TC_T2"])

In [314]:
#sns.histplot(rp["USO_LINEA_TOTAL_TC_T2_ls"]) #bins=5, color='orange'

In [315]:
#sns.kdeplot(rp, x="USO_LINEA_TOTAL_TC_T2",hue="FLAG_VENTA",fill=True,common_norm=False,alpha=0.1)

**USO_TRIM_LINEA_BBVA**

**SDO_CONSUMO_T2**

**RESENCIA_OFERTA_PLD_RENOVADO**

**ANTIGUEDAD_MES**

In [316]:
# Calcular el límite superior del boxplot
q3 = rp["ANTIGUEDAD_MES"].quantile(0.75)
q1=rp["ANTIGUEDAD_MES"].quantile(0.25)
iqr = q3 - q1
limite_superior_4 = q3 + 1.5 * iqr
limite_superior_4

384.0

In [317]:
# Remplazamos los outliers por el limite superior de su boxplot
rp["ANTIGUEDAD_MES_ls"]=rp["ANTIGUEDAD_MES"]
rp.loc[rp["ANTIGUEDAD_MES_ls"]>limite_superior_4,"ANTIGUEDAD_MES_ls"]=limite_superior_4

In [318]:
media=rp["ANTIGUEDAD_MES_ls"].mean()
print("mean: ",media)
mediana=rp["ANTIGUEDAD_MES_ls"].median()
print("mediana: ", mediana)

mean:  124.29818970932557
mediana:  106.0


In [319]:
# Reemplaza los valores faltantes con la media
rp["ANTIGUEDAD_MES_ls"].fillna(media,inplace=True)

**SUELDO_ESTIMADO**

In [320]:
# Calcular el límite superior del boxplot
q3 = rp["SUELDO_ESTIMADO"].quantile(0.75)
q1=rp["SUELDO_ESTIMADO"].quantile(0.25)
iqr = q3 - q1
limite_superior_5 = q3 + 1.5 * iqr
limite_superior_5

10722.265

In [321]:
# Remplazamos los outliers por el limite superior de su boxplot
rp["SUELDO_ESTIMADO_ls"]=rp["SUELDO_ESTIMADO"]
rp.loc[rp["SUELDO_ESTIMADO_ls"]>limite_superior_5,"SUELDO_ESTIMADO_ls"]=limite_superior_5

In [322]:
media=rp["SUELDO_ESTIMADO_ls"].mean()
print("mean: ",media)
mediana=rp["SUELDO_ESTIMADO_ls"].median()
print("mediana: ", mediana)

mean:  4187.895008032541
mediana:  3178.12


In [323]:
# Reemplaza los valores faltantes con la media
rp["SUELDO_ESTIMADO_ls"].fillna(media,inplace=True)

**EDAD**

In [324]:
# Su coefeciente de variabilidad es menor al 50%

media=rp["EDAD"].mean()
rp["EDAD_ls"]=rp["EDAD"]
rp["EDAD_ls"].fillna(media,inplace=True)

In [325]:
rp.isna().sum()

MES                                       0
CLIENTE                                   0
LINEA_RENOVADO                            0
PLAZO_RENOVADO                            0
FLAG_VENTA                                0
USO_LINEA_TOTAL_TC_T2                 19192
USO_TRIM_LINEA_BBVA                   19192
NR_ENTIDADES_TOTAL_T2                     0
DIFF_NRO_ENTIDA_TOTALES_T2_T12            0
SDO_CONSUMO_T2                         5186
RESENCIA_OFERTA_PLD_RENOVADO          13856
Ahorro_Sldo_Bco_T1                        0
PConsumo_Sldo_Bco_T1                      0
SDO_BCO_tot_sm_pasivo_Bco_6M              0
EDAD                                      1
SEXO                                      1
EST_CIVIL                                 1
ANTIGUEDAD_MES                            1
REGION                                   18
FLAG_LIMA_PROVINCIA                       0
SUELDO_ESTIMADO                          37
CUBRIR_DEUDA_CONSUMO_SF_RENOVA_PLD        0
ANTIGUEDAD_MES_ls               

# COLUMNAS CATEGORICAS

**sexo**

In [326]:
rp["SEXO_ls"] = rp["SEXO"]
rp.dropna(subset=["SEXO_ls"], inplace=True)
print(rp["SEXO_ls"].isna().sum())

0


**EST_CIVIL**

In [327]:
rp["EST_CIVIL_ls"] = rp["EST_CIVIL"]
rp.dropna(subset=["EST_CIVIL_ls"], inplace=True)
print(rp["EST_CIVIL_ls"].isna().sum())

0


**REGION**

In [328]:
rp["REGION_ls"] = rp["REGION"]
rp.dropna(subset=["REGION_ls"], inplace=True)
print(rp["REGION_ls"].isna().sum())

0


In [329]:
rp.isna().sum()

MES                                       0
CLIENTE                                   0
LINEA_RENOVADO                            0
PLAZO_RENOVADO                            0
FLAG_VENTA                                0
USO_LINEA_TOTAL_TC_T2                 19189
USO_TRIM_LINEA_BBVA                   19189
NR_ENTIDADES_TOTAL_T2                     0
DIFF_NRO_ENTIDA_TOTALES_T2_T12            0
SDO_CONSUMO_T2                         5186
RESENCIA_OFERTA_PLD_RENOVADO          13855
Ahorro_Sldo_Bco_T1                        0
PConsumo_Sldo_Bco_T1                      0
SDO_BCO_tot_sm_pasivo_Bco_6M              0
EDAD                                      0
SEXO                                      0
EST_CIVIL                                 0
ANTIGUEDAD_MES                            0
REGION                                    0
FLAG_LIMA_PROVINCIA                       0
SUELDO_ESTIMADO                          37
CUBRIR_DEUDA_CONSUMO_SF_RENOVA_PLD        0
ANTIGUEDAD_MES_ls               

# GUARDAR COLUMNAS LIMPIAS

In [331]:
rp.columns

Index(['MES', 'CLIENTE', 'LINEA_RENOVADO', 'PLAZO_RENOVADO', 'FLAG_VENTA',
       'USO_LINEA_TOTAL_TC_T2', 'USO_TRIM_LINEA_BBVA', 'NR_ENTIDADES_TOTAL_T2',
       'DIFF_NRO_ENTIDA_TOTALES_T2_T12', 'SDO_CONSUMO_T2',
       'RESENCIA_OFERTA_PLD_RENOVADO', 'Ahorro_Sldo_Bco_T1',
       'PConsumo_Sldo_Bco_T1', 'SDO_BCO_tot_sm_pasivo_Bco_6M', 'EDAD', 'SEXO',
       'EST_CIVIL', 'ANTIGUEDAD_MES', 'REGION', 'FLAG_LIMA_PROVINCIA',
       'SUELDO_ESTIMADO', 'CUBRIR_DEUDA_CONSUMO_SF_RENOVA_PLD',
       'ANTIGUEDAD_MES_ls', 'SUELDO_ESTIMADO_ls', 'EDAD_ls', 'SEXO_ls',
       'EST_CIVIL_ls', 'REGION_ls'],
      dtype='object')

In [332]:
#eliminamos cliente,
columnas=['MES', 'LINEA_RENOVADO', 'PLAZO_RENOVADO', 'FLAG_VENTA',
       'USO_LINEA_TOTAL_TC_T2', 'USO_TRIM_LINEA_BBVA', 'NR_ENTIDADES_TOTAL_T2',
       'DIFF_NRO_ENTIDA_TOTALES_T2_T12', 'SDO_CONSUMO_T2',
       'RESENCIA_OFERTA_PLD_RENOVADO', 'Ahorro_Sldo_Bco_T1',
       'PConsumo_Sldo_Bco_T1', 'SDO_BCO_tot_sm_pasivo_Bco_6M', 'FLAG_LIMA_PROVINCIA',
       'CUBRIR_DEUDA_CONSUMO_SF_RENOVA_PLD',
       'ANTIGUEDAD_MES_ls', 'SUELDO_ESTIMADO_ls', 'EDAD_ls', 'SEXO_ls',
       'EST_CIVIL_ls', 'REGION_ls']

In [333]:
rp=rp[columnas]

In [335]:
rp.to_excel('rp.xlsx', index=False)

# 2.2.- TRANSFORMACION DE DATOS

In [336]:
rp_=pd.read_excel("/content/rp.xlsx")

In [338]:
le = LabelEncoder()

rp_["SEXO_ls"] = le.fit_transform(rp_["SEXO_ls"])
rp_["EST_CIVIL_ls"] = le.fit_transform(rp_["EST_CIVIL_ls"])
rp_["REGION_ls"] = le.fit_transform(rp_["REGION_ls"])

rp_.head()

,MES,LINEA_RENOVADO,PLAZO_RENOVADO,FLAG_VENTA,USO_LINEA_TOTAL_TC_T2,USO_TRIM_LINEA_BBVA,NR_ENTIDADES_TOTAL_T2,DIFF_NRO_ENTIDA_TOTALES_T2_T12,SDO_CONSUMO_T2,RESENCIA_OFERTA_PLD_RENOVADO,...,PConsumo_Sldo_Bco_T1,SDO_BCO_tot_sm_pasivo_Bco_6M,FLAG_LIMA_PROVINCIA,CUBRIR_DEUDA_CONSUMO_SF_RENOVA_PLD,ANTIGUEDAD_MES_ls,SUELDO_ESTIMADO_ls,EDAD_ls,SEXO_ls,EST_CIVIL_ls,REGION_ls
0,201506,3770,12,0,0.000000,0.000000,1,-1,271.12,21.0,...,457,3754.333333,1,0.071915,23,3531.0,25,1,2,6
1,201506,1300,6,0,0.000000,0.000000,1,0,11200.00,9.0,...,11200,1581.333333,1,8.615380,56,1322.0,33,1,2,4
2,201506,7800,36,0,0.429507,0.225615,5,-2,20156.45,NaN,...,4778,10241.000000,1,2.584160,20,4234.0,26,1,2,5
3,201506,6630,12,0,0.000000,0.000000,1,1,20485.14,20.0,...,0,5797.333333,0,3.089760,166,6647.0,51,1,2,9
4,201506,15497,36,1,NaN,NaN,1,0,1682.83,11.0,...,1935,16755.333330,0,0.108590,55,2094.0,35,1,0,9


In [339]:
rp_["MES"].unique()

array([201506, 201505, 201507, 201508, 201509, 201504, 201503, 201502,
       201501])

In [340]:
rp_["MES"].dtype

dtype('int64')

In [341]:
# Convertir a cadena para usar "str"
rp_["MES"] = rp_["MES"].astype(str)

# Extraer el ultimo valor
rp_["MES"]=rp_["MES"].str[5:]

In [342]:
rp_["MES"].unique()

array(['6', '5', '7', '8', '9', '4', '3', '2', '1'], dtype=object)

In [343]:
rp_.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87538 entries, 0 to 87537
Data columns (total 21 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   MES                                 87538 non-null  object 
 1   LINEA_RENOVADO                      87538 non-null  int64  
 2   PLAZO_RENOVADO                      87538 non-null  int64  
 3   FLAG_VENTA                          87538 non-null  int64  
 4   USO_LINEA_TOTAL_TC_T2               68349 non-null  float64
 5   USO_TRIM_LINEA_BBVA                 68349 non-null  float64
 6   NR_ENTIDADES_TOTAL_T2               87538 non-null  int64  
 7   DIFF_NRO_ENTIDA_TOTALES_T2_T12      87538 non-null  int64  
 8   SDO_CONSUMO_T2                      82352 non-null  float64
 9   RESENCIA_OFERTA_PLD_RENOVADO        73683 non-null  float64
 10  Ahorro_Sldo_Bco_T1                  87538 non-null  int64  
 11  PConsumo_Sldo_Bco_T1                87538

In [344]:
# Asegurarse que todas las columnas sean numericas
rp_["MES"] = rp_["MES"].astype(int)
rp_["MES"].dtype

dtype('int64')

In [345]:
rp_.head(2)

,MES,LINEA_RENOVADO,PLAZO_RENOVADO,FLAG_VENTA,USO_LINEA_TOTAL_TC_T2,USO_TRIM_LINEA_BBVA,NR_ENTIDADES_TOTAL_T2,DIFF_NRO_ENTIDA_TOTALES_T2_T12,SDO_CONSUMO_T2,RESENCIA_OFERTA_PLD_RENOVADO,...,PConsumo_Sldo_Bco_T1,SDO_BCO_tot_sm_pasivo_Bco_6M,FLAG_LIMA_PROVINCIA,CUBRIR_DEUDA_CONSUMO_SF_RENOVA_PLD,ANTIGUEDAD_MES_ls,SUELDO_ESTIMADO_ls,EDAD_ls,SEXO_ls,EST_CIVIL_ls,REGION_ls
0,6,3770,12,0,0.0,0.0,1,-1,271.12,21.0,...,457,3754.333333,1,0.071915,23,3531.0,25,1,2,6
1,6,1300,6,0,0.0,0.0,1,0,11200.00,9.0,...,11200,1581.333333,1,8.615380,56,1322.0,33,1,2,4


In [346]:
rp_.isna().sum()

MES                                       0
LINEA_RENOVADO                            0
PLAZO_RENOVADO                            0
FLAG_VENTA                                0
USO_LINEA_TOTAL_TC_T2                 19189
USO_TRIM_LINEA_BBVA                   19189
NR_ENTIDADES_TOTAL_T2                     0
DIFF_NRO_ENTIDA_TOTALES_T2_T12            0
SDO_CONSUMO_T2                         5186
RESENCIA_OFERTA_PLD_RENOVADO          13855
Ahorro_Sldo_Bco_T1                        0
PConsumo_Sldo_Bco_T1                      0
SDO_BCO_tot_sm_pasivo_Bco_6M              0
FLAG_LIMA_PROVINCIA                       0
CUBRIR_DEUDA_CONSUMO_SF_RENOVA_PLD        0
ANTIGUEDAD_MES_ls                         0
SUELDO_ESTIMADO_ls                        0
EDAD_ls                                   0
SEXO_ls                                   0
EST_CIVIL_ls                              0
REGION_ls                                 0
dtype: int64

In [347]:
rp_.to_excel('rp_ml.xlsx', index=False)

# 3.- DIVISION DE DATOS PARA ENTRENAMIENTO Y PRUEBAS

In [348]:
rp_ml=pd.read_excel("/content/rp_ml.xlsx")
rp_ml

,MES,LINEA_RENOVADO,PLAZO_RENOVADO,FLAG_VENTA,USO_LINEA_TOTAL_TC_T2,USO_TRIM_LINEA_BBVA,NR_ENTIDADES_TOTAL_T2,DIFF_NRO_ENTIDA_TOTALES_T2_T12,SDO_CONSUMO_T2,RESENCIA_OFERTA_PLD_RENOVADO,...,PConsumo_Sldo_Bco_T1,SDO_BCO_tot_sm_pasivo_Bco_6M,FLAG_LIMA_PROVINCIA,CUBRIR_DEUDA_CONSUMO_SF_RENOVA_PLD,ANTIGUEDAD_MES_ls,SUELDO_ESTIMADO_ls,EDAD_ls,SEXO_ls,EST_CIVIL_ls,REGION_ls
0,6,3770,12,0,0.000000,0.000000,1,-1,271.12,21.0,...,457,3754.333333,1,0.071915,23,3531.000,25,1,2,6
1,6,1300,6,0,0.000000,0.000000,1,0,11200.00,9.0,...,11200,1581.333333,1,8.615380,56,1322.000,33,1,2,4
2,6,7800,36,0,0.429507,0.225615,5,-2,20156.45,NaN,...,4778,10241.000000,1,2.584160,20,4234.000,26,1,2,5
3,6,6630,12,0,0.000000,0.000000,1,1,20485.14,20.0,...,0,5797.333333,0,3.089760,166,6647.000,51,1,2,9
4,6,15497,36,1,NaN,NaN,1,0,1682.83,11.0,...,1935,16755.333330,0,0.108590,55,2094.000,35,1,0,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87533,8,32500,36,0,0.225158,0.448000,1,0,20450.26,15.0,...,35820,6914.000000,1,0.629238,43,3277.000,25,1,2,6
87534,7,19500,18,0,0.133990,0.096868,3,-1,22128.11,NaN,...,11880,5740.333333,1,1.134770,203,9804.000,48,1,0,6
87535,6,20800,24,0,NaN,NaN,2,0,0.00,NaN,...,12688,1717.166666,1,0.000000,149,10722.265,35,0,0,5
87536,8,22816,36,0,0.015731,0.084942,1,0,16716.47,NaN,...,33950,291.750000,0,0.732664,33,1500.000,40,1,2,12


In [374]:
rp_ml.isna().sum()

MES                                       0
LINEA_RENOVADO                            0
PLAZO_RENOVADO                            0
FLAG_VENTA                                0
USO_LINEA_TOTAL_TC_T2                 19189
USO_TRIM_LINEA_BBVA                   19189
NR_ENTIDADES_TOTAL_T2                     0
DIFF_NRO_ENTIDA_TOTALES_T2_T12            0
SDO_CONSUMO_T2                         5186
RESENCIA_OFERTA_PLD_RENOVADO          13855
Ahorro_Sldo_Bco_T1                        0
PConsumo_Sldo_Bco_T1                      0
SDO_BCO_tot_sm_pasivo_Bco_6M              0
FLAG_LIMA_PROVINCIA                       0
CUBRIR_DEUDA_CONSUMO_SF_RENOVA_PLD        0
ANTIGUEDAD_MES_ls                         0
SUELDO_ESTIMADO_ls                        0
EDAD_ls                                   0
SEXO_ls                                   0
EST_CIVIL_ls                              0
REGION_ls                                 0
dtype: int64

In [375]:
rp_ml["FLAG_VENTA"].value_counts()/len(rp_ml)*100

0    96.01316
1     3.98684
Name: FLAG_VENTA, dtype: float64

In [376]:
X = rp_ml.drop("FLAG_VENTA", axis=1)
y = rp_ml["FLAG_VENTA"]

In [377]:
# Nuestros datos estan desequilibrados, aplicamos la tecnica de muestreo "oversampling"
from imblearn.over_sampling import RandomOverSampler

oversample = RandomOverSampler(sampling_strategy='minority')
X_over, y_over = oversample.fit_resample(X, y)
y_over.value_counts()

0    84048
1    84048
Name: FLAG_VENTA, dtype: int64

In [378]:
# Divide el conjunto de datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_over, y_over, test_size=0.2, random_state=42)

# 4.- PREPARAR, ENTRENAR Y EVALUAR EL MODELO

In [379]:
# Importamos el clsificador
xgbc = XGBClassifier(random_state=42)

# Entrena el modelo en el conjunto de entrenamiento
xgbc.fit(X_train, y_train)

print("accuracy on training set: ", xgbc.score(X_train, y_train))
print("accuracy on test set: ", xgbc.score(X_test, y_test))

accuracy on training set:  0.918505904399298
accuracy on test set:  0.9009220701963118


In [380]:
confusion_matrix(y_test, xgbc.predict(X_test))

array([[14251,  2428],
       [  903, 16038]])

In [381]:
from sklearn import metrics

print("Accuracy: ", metrics.accuracy_score(y_test, xgbc.predict(X_test)))
print("Sensitivity_recall: ", metrics.recall_score(y_test, xgbc.predict(X_test)))
print("Precision: ", metrics.precision_score(y_test, xgbc.predict(X_test)))
print("F1_score: ", metrics.f1_score(y_test, xgbc.predict(X_test)))

Accuracy:  0.9009220701963118
Sensitivity_recall:  0.9466973614308483
Precision:  0.8685151088486949
F1_score:  0.9059225576863331
